In [1]:
import torch
import pytorch_lightning as pl
from tqdm import tqdm

from BridgeNetwork import BridgeSupervised, BridgeBase
from BridgeDataModule import BridgeDataModule

In [2]:
model = BridgeBase(inner_dim=1024)
state_dict = torch.load(r'../logs/base/checkpoints/bridge-base-epoch=4.ckpt')
model.load_state_dict(state_dict['state_dict'])

<All keys matched successfully>

In [3]:
datamodule = BridgeDataModule(data_dir='../data', batch_size=512, prefetch=2048, workers=4)
datamodule.setup()
loader = datamodule.val_dataloader()

In [4]:
%%capture
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
model.to(device)

In [5]:
MIN_ACTION = 52
accs = []

for batch in tqdm(loader):
    x, y = batch['observation'].to(device), batch['labels'].to(device)
    y += MIN_ACTION
    yhat = model(x)
    accs.append(model.metrics_fn(yhat,y)['acc'].item())

100%|██████████| 20/20 [14:46<00:00, 44.32s/it]


In [6]:
accs = torch.tensor(accs)
accs.mean()

tensor(0.4747)